### Imports

In [ ]:
from numpy import argmax
from pandas import read_csv
from pandas import read_json
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from nltk.stem.porter import *
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import LSTM as LSTM
import requests
import numpy as np
import nltk
import json
import spacy
import tensorflow as tf

spacy_nlp = spacy.load("en_core_web_sm")

### Stemming

In [ ]:
# Testando PorterStemmer
nltk_PS = PorterStemmer()
print(df)
print('----------------------------------------------------------------------------')

nltk_PS.stem(df.text[0])

NameError: ignored

### Extração e lista de palavras 

In [ ]:
vocabulario = dict()
# formato do vocabulário (é um objeto):
# palavra: aparicoes totais (int), qual frase apareceu (array de int), pair: (contador_frase (int), posicao (int))

contador_frase = 0
for frase in df.text.values:
  posicao = 0
  
  for s in spacy_nlp(frase):
    if s.text in vocabulario:
      vocabulario[s.text][0] += 1
      vocabulario[s.text][1].append(contador_frase)
      vocabulario[s.text][2].append((contador_frase, posicao))
    else:
      vocabulario.update({s.text: [0, [], []]})
    posicao += 1

  contador_frase += 1
print(vocabulario)

# Lista de palavras
lista = list(vocabulario)
print(len(vocabulario))

### Tensorflow/Keras

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf

train, test = train_test_split(df)
train, test = train_test_split(df.values)
#X_train = list(train)
#X_train, y_train = train.text.values, train['category (Intent)'].values
#X_test, y_test = test.text.values, test['category (Intent)'].values
#print(y_train)
# print(f'Train \n{train}\nSize:{len(train)}\nTest \n{test}\nSize:{len(test)}\n')

In [ ]:
encoder = tf.keras.layers.experimental.preprocessing.TextVectorization(max_tokens=len(df))
encoder.adapt(df.text.values)
print(encoder.vocabulary_size())
# print(encoder)
# encoder.adapt(.map(lambda text, label: text))

### LSTM

##### Fonte & documentação: [Keras - LSTM](https://keras.io/api/layers/recurrent_layers/lstm/)

In [ ]:

vocab_size = 392

df = read_csv('http://bolao.lmvdigital.com.br/testFiltered.csv', header=0)
df_train = read_csv('http://bolao.lmvdigital.com.br/trainFiltered.csv', header=0)
X_train = df_train.text
y_train = df_train.category

X_test = df.text
y_test = df['category (Intent)']


X_train = [one_hot(d, vocab_size,filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~',lower=True, split=' ') for d in X_train]
X_test = [one_hot(d, vocab_size,filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~',lower=True, split=' ') for d in X_test]
y_train = [one_hot(d, vocab_size,filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~',lower=True, split=' ') for d in y_train]
y_test = [one_hot(d, vocab_size,filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~',lower=True, split=' ') for d in y_test]

max_length = 30
X_train = pad_sequences(X_train, maxlen=max_length, padding='pre')
X_test = pad_sequences(X_test, maxlen=max_length, padding='pre')
y_train = pad_sequences(y_train, maxlen=4, padding='pre')
y_test = pad_sequences(y_test, maxlen=4, padding='pre')

# print(X_train, X_test, y_train, y_test)
# print(y_test)
lstm = LSTM(4)
o = lstm(y_test)
print(o.shape)
# whole_seq_output, final_memory_state, final_carry_state = outputLSTM(y_test)

### Funcionando

In [ ]:
vocab_size = 500

df_train = read_csv('http://bolao.lmvdigital.com.br/trainFiltered.csv', header=0)
X_train = df_train.text
y_train = df_train.category

X_test = df.text
y_test = df['category (Intent)']


X_train = [one_hot(d, vocab_size,filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~',lower=True, split=' ') for d in X_train]
X_test = [one_hot(d, vocab_size,filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~',lower=True, split=' ') for d in X_test]
y_train = [one_hot(d, vocab_size,filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~',lower=True, split=' ') for d in y_train]
y_test = [one_hot(d, vocab_size,filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~',lower=True, split=' ') for d in y_test]


"I am hungry"
[0, 0, 0, 10, 2, 30]

max_length = 30
X_train = pad_sequences(X_train, maxlen=max_length, padding='pre')
X_test = pad_sequences(X_test, maxlen=max_length, padding='pre')
y_train = pad_sequences(y_train, maxlen=4, padding='pre')
y_test = pad_sequences(y_test, maxlen=4, padding='pre')

model = Sequential()
model.add(Embedding(vocab_size, 8, input_length=50))
model.add(Flatten())
model.add(Dense(4, activation='softmax'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

model.fit(X_train, y_train, epochs=20, verbose=0)

loss, accuracy = model.evaluate(X_train, y_train, verbose=1)
print('Training Accuracy is {}'.format(accuracy*100))

loss, accuracy = model.evaluate(X_test,y_test)
print('Testing Accuracy is {} '.format(accuracy*100))

In [ ]:

print(X_train, y_train)

## Text vectorization


In [ ]:
vocab_size = 393
df_train = read_csv('http://bolao.lmvdigital.com.br/trainFiltered.csv', header=0)
X_train = df_train.text
y_train = df_train.category

X_test = df.text
y_test = df['category (Intent)']


y_train = [one_hot(d, vocab_size,filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~',lower=True, split=' ') for d in y_train]
y_train = pad_sequences(y_train, maxlen=4, padding='pre')
y_test = [one_hot(d, vocab_size,filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~',lower=True, split=' ') for d in y_test]
y_test = pad_sequences(y_test, maxlen=4, padding='pre')

# print(X_train)
encoder = tf.keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size)
encoder.adapt(list(X_train.values))
#intent_encoder = tf.keras.layers.experimental.preprocessing.TextVectorization(max_tokens=6)
#intent_encoder.adapt(list(y_train.values))
#pivot = []
#[pivot.append(intent_encoder(f)[0]) for f in y_train]
#y_train = pivot.copy()
# vocab = np.array(encoder.get_vocabulary())
# vocab[:10]
# encoded_example = encoder(X_train[0])
# print(encoded_example, X_train[0])

In [ ]:
model = Sequential([
    encoder,
    Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=64
    ),
    Bidirectional(LSTM(64)),
    Dense(4, activation='softmax')
])

model.compile(loss=tf.keras.losses.BinaryCrossentropy(),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])
print(len(X_train), len(y_train))
model.fit(X_train, y_train, 10, epochs=20)

loss, accuracy = model.evaluate(X_train, y_train, verbose=1)
print('Training Accuracy is {}'.format(accuracy*100))

In [ ]:
loss, accuracy = model.evaluate(X_train, y_train, verbose=1)
print('Training Accuracy is {}'.format(accuracy*100))

loss, accuracy = model.evaluate(X_test,y_test)
print('Testing Accuracy is {} '.format(accuracy*100))
sample_text = ('I want a new card')
predictions = model.predict(np.array([sample_text]))
print(predictions)

In [ ]:
vocab_size = 392

df = read_csv('http://bolao.lmvdigital.com.br/testFiltered.csv', header=0)
df_train = read_csv('http://bolao.lmvdigital.com.br/trainFiltered.csv', header=0)
X_train = df_train.text
y_train = df_train.category

X_test = df.text
y_test = df['category (Intent)']

X_train = [one_hot(d, vocab_size,filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~',lower=True, split=' ') for d in X_train]
X_test = [one_hot(d, vocab_size,filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~',lower=True, split=' ') for d in X_test]
y_train = [one_hot(d, vocab_size) for d in y_train]
y_test = [one_hot(d, vocab_size) for d in y_test]

max_length = 30
X_train = pad_sequences(X_train, maxlen=max_length, padding='pre')
X_test = pad_sequences(X_test, maxlen=max_length, padding='pre')
y_train = pad_sequences(y_train, maxlen=1, padding='pre')
y_test = pad_sequences(y_test, maxlen=1, padding='pre')


#Import Random Forest Model
from sklearn.ensemble import RandomForestClassifier

#Create a Gaussian Classifier
clf=RandomForestClassifier(n_estimators=100)

#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train,y_train)

y_pred=clf.predict(X_test)

#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

In [ ]:
df = read_csv('http://bolao.lmvdigital.com.br/NLU.csv', header=0)
# df

X_train, X_test, y_train, y_test = train_test_split(df.text, df.intent,test_size=0.3)

encoder = tf.keras.layers.experimental.preprocessing.TextVectorization(max_tokens=len(df))
encoder.adapt(df.text.values)
vocab_size = encoder.vocabulary_size()
print(encoder.vocabulary_size())

y_train = [one_hot(d, vocab_size,filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~',lower=True, split=' ') for d in y_train]
y_train = pad_sequences(y_train, maxlen=4, padding='pre')
y_test = [one_hot(d, vocab_size,filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~',lower=True, split=' ') for d in y_test]
y_test = pad_sequences(y_test, maxlen=4, padding='pre')

# print(X_train)
encoder = tf.keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size)
encoder.adapt(list(X_train.values))

model = Sequential([
    encoder,
    Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=64
    ),
    Bidirectional(LSTM(64)),
    Dense(4, activation='softmax')
])

model.compile(loss=tf.keras.losses.BinaryCrossentropy(),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])
print(len(X_train), len(y_train))
model.fit(X_train, y_train, 10, epochs=5)

loss, accuracy = model.evaluate(X_train, y_train, verbose=1)
print('Training Accuracy is {}'.format(accuracy*100))

X_train, X_test, y_train, y_test = train_test_split(df.text, df.intent,test_size=0.3)

X_train = [one_hot(d, vocab_size,filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~',lower=True, split=' ') for d in X_train]
X_test = [one_hot(d, vocab_size,filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~',lower=True, split=' ') for d in X_test]
y_train = [one_hot(d, vocab_size) for d in y_train]
y_test = [one_hot(d, vocab_size) for d in y_test]

max_length = 30
X_train = pad_sequences(X_train, maxlen=max_length, padding='pre')
X_test = pad_sequences(X_test, maxlen=max_length, padding='pre')
y_train = pad_sequences(y_train, maxlen=1, padding='pre')
y_test = pad_sequences(y_test, maxlen=1, padding='pre')
from sklearn.ensemble import RandomForestClassifier
clf=RandomForestClassifier(n_estimators=100)
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)
from sklearn import metrics
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

533
14219 14219
Epoch 1/5
1422/1422 [==============================] - 25s 15ms/step - loss: -1099.5378 - accuracy: 0.9411
Epoch 2/5
1422/1422 [==============================] - 22s 15ms/step - loss: -2233.5571 - accuracy: 0.9411
Epoch 3/5
1422/1422 [==============================] - 22s 16ms/step - loss: -3222.2964 - accuracy: 0.9411
Epoch 4/5
1422/1422 [==============================] - 22s 15ms/step - loss: -4208.9756 - accuracy: 0.9411
Epoch 5/5
445/445 [==============================] - 3s 6ms/step - loss: -5691.7061 - accuracy: 0.9411
Training Accuracy is 94.11351084709167


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:53: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


Accuracy: 0.992615687561536


In [ ]:
sample_text = ('Where would you like to book your hotel? I want a hotel room with Bolivia view')
predictions = model.predict(np.array([sample_text]))
print(predictions)

[[0.0000000e+00 0.0000000e+00 2.8077792e-16 1.0000000e+00]]


In [ ]:
# https://stackoverflow.com/questions/38971293/get-class-labels-from-keras-functional-model
y_prob = model.predict(np.array([sample_text]))
y_classes = y_prob.argmax(axis=-1)
print(y_classes[0])

3


## Testando o que Mendonça falou sobre o parâmetro alimentado ao Dense ser o número de intents que a gente tem no dataset


In [ ]:
from pandas import DataFrame
df = read_csv('http://bolao.lmvdigital.com.br/NLU.csv', header=0)
# Aplica funções lambda para separar pergunta?resposta e intent+intent
# df.text = df.text.apply(lambda x: x.split('?')[1])
# df.intent = df.intent.apply(lambda x: x.split('+')[1] if len(x.split('+'))>1 else x.split('+')[0])
print('Intents: ', df.intent.nunique())
# df
frase = None

def codigoAcima(intents):
  X_train, X_test, y_train, y_test = train_test_split(df.text, df.intent,test_size=0.3)

  encoder = tf.keras.layers.experimental.preprocessing.TextVectorization(max_tokens=len(df))
  encoder.adapt(df.text.values)
  vocab_size = encoder.vocabulary_size()
  print(encoder.vocabulary_size())

  print('y_train puro:\n', y_train)
  bkp_y_train = y_train
  y_train = [one_hot(d, vocab_size,filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~',lower=True, split=' ') for d in y_train]
  print('y_train one_hot:', y_train[-5:], '...]]')
  mapeamento = []
  print('\n\n\n\n', y_train)
  mapeamento = list(map(lambda x: (y_train[x], bkp_y_train.values[x]), range(len(y_train))))
  dict = {}
  for i in range(len(y_train)):
    dict[str(y_train[i])] = bkp_y_train.values[i]
  y_train = pad_sequences(y_train, maxlen=intents, padding='pre')
  print('y_train pad_sequences:\n', y_train)
  y_test = [one_hot(d, vocab_size,filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~',lower=True, split=' ') for d in y_test]
  y_test = pad_sequences(y_test, maxlen=intents, padding='pre')


  # print(*dir(y_train), sep='\n')
  # print(y_train, y_train.view(), y_train.var())
  # return
  # print(X_train)
  
  encoder = tf.keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size)
  encoder.adapt(list(X_train.values))

  model = Sequential([
      encoder,
      Embedding(
          input_dim=len(encoder.get_vocabulary()),
          output_dim=64
      ),
      Bidirectional(LSTM(64)),
      Dense(intents, activation='softmax')
  ])

  model.compile(loss=tf.keras.losses.BinaryCrossentropy(),
                optimizer=tf.keras.optimizers.Adam(1e-4),
                metrics=['accuracy'])
  print(len(X_train), len(y_train))
  model.fit(X_train, y_train, epochs=20)

  loss, accuracy = model.evaluate(X_train, y_train, verbose=1)
  print('Training Accuracy is {}'.format(accuracy*100))

  loss, accuracy = model.evaluate(X_test, y_test, verbose=1)
  print('Test Accuracy is {}'.format(accuracy*100))

  # Random Forest's garbage
  # X_train, X_test, y_train, y_test = train_test_split(df.text, df.intent,test_size=0.3)
  # X_train = [one_hot(d, vocab_size,filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~',lower=True, split=' ') for d in X_train]
  # X_test = [one_hot(d, vocab_size,filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~',lower=True, split=' ') for d in X_test]
  # y_train = [one_hot(d, vocab_size) for d in y_train]
  # y_test = [one_hot(d, vocab_size) for d in y_test]
  # max_length = 30
  # X_train = pad_sequences(X_train, maxlen=max_length, padding='pre')
  # X_test = pad_sequences(X_test, maxlen=max_length, padding='pre')
  # y_train = pad_sequences(y_train, maxlen=1, padding='pre')
  # y_test = pad_sequences(y_test, maxlen=1, padding='pre')
  # from sklearn.ensemble import RandomForestClassifier
  # clf=RandomForestClassifier(n_estimators=100)
  # clf.fit(X_train,y_train)
  # y_pred=clf.predict(X_test)
  # from sklearn import metrics
  # print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

  sample_text = ('I want a hotel room with Recife view')
  predictions = model.predict([sample_text])
  frase = pad_sequences([one_hot(sample_text, vocab_size, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True, split=' ')], maxlen=1, padding='pre')
  print(dict)
  for f in frase:
    #print(frase, mapeamento[frase[f][0]])
    print(frase, f)
  # predictions = model.predict(frase)
  # print(predictions)

codigoAcima(df.intent.nunique())


Intents:  7
533
y_train puro:
 3438            HotelSearch
13807              Negation
2034            HotelSearch
10666    HotelSearch+Filter
4675            HotelSearch
                ...        
10076    HotelSearch+Filter
14431              Negation
15041              Negation
1028            HotelSearch
14758              Negation
Name: intent, Length: 14219, dtype: object
y_train one_hot: [[121, 295], [389], [389], [121], [389]] ...]]




 [[121], [389], [121], [121, 295], [121], [121, 295], [389], [389], [121, 295], [389], [389], [121], [121], [121], [121], [121, 295], [121], [121, 295], [389], [389], [121, 295], [121], [121], [389], [121, 295], [121], [121], [121], [376], [376], [376], [161], [121], [389], [121, 295], [121, 295], [389], [376], [389], [121], [121], [389], [121], [376], [121], [121], [121], [121], [121], [121], [389], [389], [121], [121, 295], [121], [121], [121, 295], [121, 295], [376, 47], [376], [376], [121], [121, 295], [389], [121], [121, 295], [376, 47], [

### (Vitor) tentando resolver problema de saber qual intent é

In [ ]:
from pandas import DataFrame
from copy import deepcopy
try:
  if(len(df) < 10000):
    pass
except:
  df = read_csv('http://bolao.lmvdigital.com.br/NLU.csv', header=0)

frase = None
mapeamento = None
encoder = None
mode = None
loss = None
acuracy = None
sample_text = None
predictions = None
print('Intents: ', df.intent.nunique())

def codigoAcima(intents):
  X_train, X_test, y_train, y_test = train_test_split(df.text, df.intent,test_size=0.3)
  encoder = tf.keras.layers.experimental.preprocessing.TextVectorization(max_tokens=len(df))
  encoder.adapt(df.text.values)
  vocab_size = encoder.vocabulary_size()
  # print('Tamanho do vocabulário: ', encoder.vocabulary_size())
  # print('y_train puro:\n', y_train)
  # bkp_y_train = deepcopy(y_train)
  bkp_y_train = y_train
  # print(type(y_train))
  y_train = [one_hot(d, vocab_size,filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~',lower=True, split=' ') for d in y_train]
  # print('y_train one_hot:', y_train[-5:], '...]]')

  mapeamento = []
  s = dict()
  print('Para intent 167: ', y_train[166], y_train[167], y_train[168])
  for y in y_train:
    if len(y)==1:
      s.update({str(y[0]): []})
    elif len(y)==2:
      s.update({str(y[0])+'+'+str(y[1]): []})

  print('Mapa de intents: ', s)
  # print('\n\n\n\n', y_train)
  # mapeamento = list(map(lambda x: (y_train[x], bkp_y_train.values[x]), range(len(y_train))))

  y_train = pad_sequences(y_train, maxlen=intents, padding='pre')
  # print('y_train pad_sequences:\n', y_train)
  y_test = [one_hot(d, vocab_size,filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~',lower=True, split=' ') for d in y_test]
  y_test = pad_sequences(y_test, maxlen=intents, padding='pre')

  encoder = tf.keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size)
  encoder.adapt(list(X_train.values))

  model = Sequential([
      encoder,
      Embedding(
          input_dim=len(encoder.get_vocabulary()),
          output_dim=64
      ),
      Bidirectional(LSTM(64)),
      Dense(intents, activation='softmax')
  ])

  model.compile(loss=tf.keras.losses.BinaryCrossentropy(),
                optimizer=tf.keras.optimizers.Adam(1e-4),
                metrics=['accuracy'])
  print(len(X_train), len(y_train))
  model.fit(X_train, y_train, epochs=1)

  loss, accuracy = model.evaluate(X_train, y_train, verbose=1)
  print('Training Accuracy is {}'.format(accuracy*100))

  loss, accuracy = model.evaluate(X_test, y_test, verbose=1)
  print('Test Accuracy is {}'.format(accuracy*100))

  sample_text = ('I want a hotel room with Ribeirão view')
  predictions = model.predict([sample_text])
  frase = pad_sequences([one_hot(sample_text, vocab_size, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True, split=' ')], maxlen=1, padding='pre')
  
  def Testes():
    frases_testes = ['I want a hotel room with Ribeirão view',
                     'How can I help you today? Book me a 5 star hotel on 19 jan near Ribeirão',
                     'How can I help you today? I would like to cancel a four rooms Duluxe room hotel in hyatt at Ribeirão',
                     'Would you like to continue ? Caruaru is confirmed Book it',
                     'Would you like to continue ? is hyatt most visited hotel in Caruaru'
                     ]
    # Esperado HotelSearch+Filter, HotelSearch, Negation, Affirmation, HotelDetail

    for sample_text in frases_testes:
      predictions = model.predict([sample_text])
      frase = pad_sequences([one_hot(sample_text, vocab_size, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True, split=' ')], maxlen=1, padding='pre')
      try:
        if len(frase)==1:
          print('1 Intent para a frase {}:\n'.format(frase), list(bkp_y_train)[frase[0][0]], list(df.text)[frase[0][0]])
        elif len(frase)==2:
          print('2 Intents para a frase {}:\n'.format(frase),list(bkp_y_train)[frase[0][0]], list(bkp_y_train)[frase[1][0]], list(df.text)[frase[0][0]], list(df.text)[frase[1][0]])
      except:
        print('Deu ruim')
        # print(list(bkp_y_train)[frase[0][0]])
        # print('Except: ', bkp_y_train[frase[0][0]])
  Testes()

codigoAcima(df.intent.nunique())

Intents:  7
Para intent 167:  [154] [154] [234]
Mapa de intents:  {'154': [], '154+359': [], '370+255': [], '370': [], '264+255': [], '264': [], '234': []}
14219 14219
445/445 [==============================] - 5s 10ms/step - loss: -793.2347 - accuracy: 0.9430
Training Accuracy is 94.30339932441711
191/191 [==============================] - 2s 9ms/step - loss: -791.0401 - accuracy: 0.9362
Test Accuracy is 93.61667037010193
1 Intent para a frase [[167]]:
 HotelSearch How can I help you today? Book me a hotel with checkin time 19 jan and checkout at 19 jan in Algeria
1 Intent para a frase [[79]]:
 HotelSearch+Filter How can I help you today? Book me a hotel today in Armenia
1 Intent para a frase [[79]]:
 HotelSearch+Filter How can I help you today? Book me a hotel today in Armenia
1 Intent para a frase [[163]]:
 HotelSearch How can I help you today? Book me a hotel with checkin time 19 jan and checkout at 19 jan in andra pradesh
1 Intent para a frase [[18]]:
 HotelSearch+Filter How can I

# Criação de um modelo sequencial, após realizar Text Vectorization, para aplicar o LSTM (com CategoricalCrossentropy, pois temos mais de uma classe), e realizando testes de predição para verificar acurácia.
###### Agora vai, confia no Pai

In [ ]:
from pandas import read_csv
from sklearn.model_selection import train_test_split
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from keras.layers.embeddings import Embedding
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import LSTM as LSTM
import tensorflow as tf

In [ ]:

df = read_csv('http://bolao.lmvdigital.com.br/NLU.csv', header=0)
num_intents = df.intent.nunique()
print('Intents: ', num_intents)
# Aplica funções lambda para separar pergunta?resposta e intent+intent
df.text = df.text.apply(lambda x: x.split('?')[1])
print("texts:\n", df.text)
print("\n\nintents:\n", df.intent)

Intents:  7
texts:
 0            Book me a 5 star hotel on 19 jan near kolkata
1          Book me a one star hotel on 19 jan near kolkata
2          Book me a two star hotel on 19 jan near kolkata
3         Book me a three star hotel on 19 jan near kol...
4         Book me a four star hotel on 19 jan near kolkata
                               ...                        
20308     Hotel Hyatt looks good can you tell me more a...
20309     Hotel Marriot looks good can you tell me more...
20310     Hotel Taj looks good can you tell me more abo...
20311     Hotel ITC Royal Bengal looks good can you tel...
20312     Hotel Padmavati looks good can you tell me mo...
Name: text, Length: 20313, dtype: object


intents:
 0        HotelSearch
1        HotelSearch
2        HotelSearch
3        HotelSearch
4        HotelSearch
            ...     
20308    HotelDetail
20309    HotelDetail
20310    HotelDetail
20311    HotelDetail
20312    HotelDetail
Name: intent, Length: 20313, dtype: object


In [ ]:
vocab = {""}
for text in df.text:
  for word in text.split():
    if word not in vocab:
      vocab.add(word)
vocab_size = len(vocab)
print("Vocab size:", len(vocab))

Vocab size: 587


In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(df.text, df.intent,test_size=0.3)

encoder = TextVectorization(max_tokens=vocab_size)
encoder.adapt(df.text.values)
vocab_size = encoder.vocabulary_size()
print(encoder.vocabulary_size())

525


In [ ]:
def get_intents(Y_train, num_intents):
  count = 0
  intent_map = {}
  intents = [""]*num_intents
  for idx, intent in enumerate(Y_train):
    if intent not in intent_map.keys():
      intents[count] = intent
      intent_map[intent] = count
      count += 1
  return (intents, intent_map)

def encode_intents(Y_train, intent_map):
  intent_list = []
  for idx, intent in enumerate(Y_train):
    intent_list.append([0]*len(intent_map))
    intent_list[idx][intent_map[intent]] = 1
  return intent_list

intents, intent_map = get_intents(Y_train, num_intents)
y_train = encode_intents(Y_train, intent_map)
y_test = encode_intents(Y_test, intent_map)
print(intents)
print(intent_map)
print(y_train)

y_train = pad_sequences(y_train, maxlen=num_intents, padding='pre')
y_test = pad_sequences(y_test, maxlen=num_intents, padding='pre')

['HotelSearch', 'Negation', 'HotelSearch+Filter', 'Affirmation', 'HotelDetail', 'Affirmation+Payment', 'Negation+Payment']
{'HotelSearch': 0, 'Negation': 1, 'HotelSearch+Filter': 2, 'Affirmation': 3, 'HotelDetail': 4, 'Affirmation+Payment': 5, 'Negation+Payment': 6}
[[1, 0, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 

In [ ]:
encoder = tf.keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size)
encoder.adapt(list(X_train.values))

model = Sequential([
    encoder,
    Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=64
    ),
    Bidirectional(LSTM(64)),
    Dense(num_intents, activation='softmax')
])

model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])
print(len(X_train), len(y_train))

14219 14219


In [ ]:
model.fit(X_train, y_train, epochs=20)

Epoch 1/20
445/445 [==============================] - 15s 25ms/step - loss: 1.2828 - accuracy: 0.5449
Epoch 2/20
445/445 [==============================] - 11s 24ms/step - loss: 0.5439 - accuracy: 0.8025
Epoch 3/20
445/445 [==============================] - 11s 24ms/step - loss: 0.3786 - accuracy: 0.8418
Epoch 4/20
445/445 [==============================] - 10s 24ms/step - loss: 0.2934 - accuracy: 0.8771
Epoch 5/20
445/445 [==============================] - 10s 24ms/step - loss: 0.2336 - accuracy: 0.9094
Epoch 6/20
445/445 [==============================] - 10s 23ms/step - loss: 0.1931 - accuracy: 0.9301
Epoch 7/20
445/445 [==============================] - 11s 24ms/step - loss: 0.1585 - accuracy: 0.9411
Epoch 8/20
445/445 [==============================] - 11s 24ms/step - loss: 0.1387 - accuracy: 0.9480
Epoch 9/20
445/445 [==============================] - 11s 24ms/step - loss: 0.1212 - accuracy: 0.9522
Epoch 10/20
445/445 [==============================] - 10s 24ms/step - loss: 0.105

In [ ]:
loss, accuracy = model.evaluate(X_train, y_train, verbose=1)
print('Training Accuracy is {}'.format(accuracy*100))

loss, accuracy = model.evaluate(X_test, y_test, verbose=1)
print('Test Accuracy is {}'.format(accuracy*100))

445/445 [==============================] - 4s 7ms/step - loss: 0.0522 - accuracy: 0.9762
Training Accuracy is 97.61586785316467
191/191 [==============================] - 1s 7ms/step - loss: 0.0871 - accuracy: 0.9595
Test Accuracy is 95.94683051109314


In [ ]:
sample_text = ('Book me a hotel today in Recife')
prediction = model.predict([sample_text])
for idx, value in enumerate(prediction[0]):
  print("{}: {:.2f}%".format(intents[idx], 100*value))

HotelSearch: 18.45%
Negation: 10.60%
HotelSearch+Filter: 50.86%
Affirmation: 7.28%
HotelDetail: 9.32%
Affirmation+Payment: 2.34%
Negation+Payment: 1.16%


In [ ]:
sample_text = ('is there any room available on 6 Jan?')
prediction = model.predict([sample_text])
for idx, value in enumerate(prediction[0]):
  print("{}: {:.2f}%".format(intents[idx], 100*value))

HotelSearch: 14.52%
Negation: 11.72%
HotelSearch+Filter: 0.71%
Affirmation: 0.93%
HotelDetail: 69.80%
Affirmation+Payment: 0.08%
Negation+Payment: 2.24%


In [ ]:
sample_text = ('Book me a 5 star hotel on 19 jan near Jaboatão dos Guararapes')
prediction = model.predict([sample_text])
for idx, value in enumerate(prediction[0]):
  print("{}: {:.2f}%".format(intents[idx], 100*value))

HotelSearch: 8.49%
Negation: 0.49%
HotelSearch+Filter: 90.55%
Affirmation: 0.16%
HotelDetail: 0.26%
Affirmation+Payment: 0.04%
Negation+Payment: 0.01%
